In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA")
else:
    device = torch.device('cpu')
    print("NO")

CUDA


In [ ]:
!pip install pandas
!pip install -U sentence-transformers
!pip install transformers[torch]
!pip install datasets

In [2]:
from sentence_transformers import SentenceTransformer
import torch
import pandas as pd
df = pd.read_csv('BigBasket Products.csv')
df["combined"] = df["product"].fillna('') + ". Description :" + df["description"].fillna('') + "!"
df

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,combined
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,Garlic Oil - Vegetarian Capsule 500 mg. Descri...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...",Water Bottle - Orange. Description :Each produ...
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...","Brass Angle Deep - Plain, No.2. Description :A..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,Cereal Flip Lid Container/Storage Jar - Assort...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,Creme Soft Soap - For Hands & Body. Descriptio...
...,...,...,...,...,...,...,...,...,...,...,...
27550,27551,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...,"Wottagirl! Perfume Spray - Heaven, Classic. De..."
27551,27552,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...,Rosemary. Description :Puramate rosemary is en...
27552,27553,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...,Peri-Peri Sweet Potato Chips. Description :We ...
27553,27554,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori...",Green Tea - Pure Original. Description :Tetley...


In [3]:
def generate_combinations(df):
    combinations = set()  # Use a set to automatically handle duplicates
    
    # Iterate through each subcategory
    for sub_category, group in df.groupby('sub_category'):
        # Get products and their combined description for the current subcategory
        products_combined = group[['product', 'combined']].set_index('product')
        
        # Get the list of products in the subcategory
        products = products_combined.index.tolist()
        
        # Generate up to 5 combinations for each product (maximum 5)
        for i in range(len(products)):
            # Limit the number of combinations to 5
            for j in range(i + 1, min(i + 6, len(products))):  # Combine with the next 5 products
                # Get the combined descriptions as strings
                prod_1 = str(products_combined.loc[products[i], 'combined'])
                prod_2 = str(products_combined.loc[products[j], 'combined'])
                
                if prod_1 != prod_2:
                    combinations.add(tuple(sorted([prod_1, prod_2])))
    
    combination_df = pd.DataFrame(list(combinations), columns=['product_1', 'product_2'])
    
    return combination_df
combination_df = generate_combinations(df)

In [4]:
combination_df

,product_1,product_2
0,"Cold Extracted Juice - Basics, Guava. Descript...",Juice - Orange Oomph. Description :B Natural M...
1,Dishwasher Rinse Aid forCrystal Shine. Descrip...,product\nGerm Kill Spray Germ Kill Spray. D...
2,Pulse Deodorant. Description :The World’s # 1 ...,product\nCool Hold Styling Hair Gel Cool Ho...
3,Morning Energy Face Wash - Aqua Splash. Descri...,product\nWhitening Even Tone Body Lotion - UV ...
4,Gas Lighter-Stainless Steel Aristo - Assorted ...,Softgrip Scissor (17cm). Description :Fackelma...
...,...,...
134912,product\nMysore Pak Mysore Pak. Description...,product\nRasogolla Rasogolla. Description :...
134913,Intensive Care Revitalizing Green Tea Body Lot...,Pigmentation Reducing Complex. Description :De...
134914,product\nChia Seeds Chia Seeds. Description...,product\nCocoa Powder - Non-Alkalised Cocoa...
134915,Palak 250g+Ladies Finger 500g+Cucumber 1kg+Cap...,Spring Onion - Chopped. Description :Spring on...


In [5]:
import pandas as pd
import random

def generate_mismatched_combinations(df):
    # Initialize a list to store mismatched combinations
    mismatched_combinations = []
    
    # Group the dataframe by sub-category
    grouped_by_sub_category = df.groupby('sub_category')
    
    # Create a dictionary to quickly access products of different sub-categories
    other_sub_category_products = {sub_cat: df[df['sub_category'] != sub_cat] for sub_cat in df['sub_category'].unique()}
    
    # Iterate through each row in the original dataframe
    for idx, row in df.iterrows():
        product = row['product']
        product_combined = str(row['combined'])
        product_sub_category = row['sub_category']
        
        # Access products from different sub-categories
        other_products_df = other_sub_category_products[product_sub_category]
        
        # Randomly sample 3 products from the other sub-categories
        sampled_other_products = other_products_df.sample(n=min(3, len(other_products_df)), random_state=42)
        
        # Create mismatched combinations
        for _, other_row in sampled_other_products.iterrows():
            other_product = other_row['product']
            other_product_combined = str(other_row['combined'])
            
            # Ensure no duplicates (same combination in reverse order)
            if product != other_product:
                mismatch_combination = tuple(sorted([product_combined, other_product_combined]))
                mismatched_combinations.append(mismatch_combination)
    
    # Convert the list of combinations to a DataFrame
    mismatched_combination_df = pd.DataFrame(mismatched_combinations, columns=['product_1', 'product_2'])
    
    return mismatched_combination_df

# Generate the mismatched combinations
mismatched_combination_df = generate_mismatched_combinations(df)

# Inspect the new DataFrame before merging it with the main DataFrame
print(mismatched_combination_df.head())


                                           product_1  \
0  Garlic Oil - Vegetarian Capsule 500 mg. Descri...   
1  Garlic Oil - Vegetarian Capsule 500 mg. Descri...   
2  Garlic Oil - Vegetarian Capsule 500 mg. Descri...   
3  Lamb & Vegetables Dog Food - Adult, All Breeds...   
4  Biryani Basmati Rice/Basmati Akki. Description...   

                                           product_2  
0  Plastic Cloth Hangers - Brown, Elegance. Descr...  
1  Snacks/Cookie Glass Jar With Steel Cap. Descri...  
2  Little Millet Khichadi Mix - Gluten Free. Desc...  
3  Water Bottle - Orange. Description :Each produ...  
4  Water Bottle - Orange. Description :Each produ...  


In [6]:
import numpy as np
mismatched_combination_df['score'] = np.random.uniform(0.10,.20, size = len(mismatched_combination_df))

In [7]:
mismatched_combination_df

,product_1,product_2,score
0,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,"Plastic Cloth Hangers - Brown, Elegance. Descr...",0.120180
1,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,Snacks/Cookie Glass Jar With Steel Cap. Descri...,0.148833
2,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,Little Millet Khichadi Mix - Gluten Free. Desc...,0.115156
3,"Lamb & Vegetables Dog Food - Adult, All Breeds...",Water Bottle - Orange. Description :Each produ...,0.122995
4,Biryani Basmati Rice/Basmati Akki. Description...,Water Bottle - Orange. Description :Each produ...,0.118265
...,...,...,...
82660,Ayurvedic Massage Oil - Dhanwantharam Oil. Des...,Green Tea - Pure Original. Description :Tetley...,0.175249
82661,Green Tea - Pure Original. Description :Tetley...,Popular Aluminium Outer Lid Pressure Cooker (1...,0.156526
82662,Beetroot - Diced. Description :These edible ru...,United Dreams Go Far Deodorant. Description :T...,0.145101
82663,Fresh Natural Women Deodorant - Long Lasting F...,United Dreams Go Far Deodorant. Description :T...,0.195931


In [8]:
mismatched_combination_df.drop(index=82663, inplace=True)

In [9]:
mismatched_combination_df

,product_1,product_2,score
0,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,"Plastic Cloth Hangers - Brown, Elegance. Descr...",0.120180
1,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,Snacks/Cookie Glass Jar With Steel Cap. Descri...,0.148833
2,Garlic Oil - Vegetarian Capsule 500 mg. Descri...,Little Millet Khichadi Mix - Gluten Free. Desc...,0.115156
3,"Lamb & Vegetables Dog Food - Adult, All Breeds...",Water Bottle - Orange. Description :Each produ...,0.122995
4,Biryani Basmati Rice/Basmati Akki. Description...,Water Bottle - Orange. Description :Each produ...,0.118265
...,...,...,...
82659,Green Tea - Pure Original. Description :Tetley...,Organic Sattu Maavu Multi-Grain Millets Porrid...,0.196773
82660,Ayurvedic Massage Oil - Dhanwantharam Oil. Des...,Green Tea - Pure Original. Description :Tetley...,0.175249
82661,Green Tea - Pure Original. Description :Tetley...,Popular Aluminium Outer Lid Pressure Cooker (1...,0.156526
82662,Beetroot - Diced. Description :These edible ru...,United Dreams Go Far Deodorant. Description :T...,0.145101


In [10]:
mismatched_combination_df = mismatched_combination_df.reset_index(drop=True) 

In [11]:
combination_df['score'] = np.random.uniform(0.87, 0.97, size = len(combination_df))

In [12]:
combination_df

,product_1,product_2,score
0,"Cold Extracted Juice - Basics, Guava. Descript...",Juice - Orange Oomph. Description :B Natural M...,0.948012
1,Dishwasher Rinse Aid forCrystal Shine. Descrip...,product\nGerm Kill Spray Germ Kill Spray. D...,0.928530
2,Pulse Deodorant. Description :The World’s # 1 ...,product\nCool Hold Styling Hair Gel Cool Ho...,0.891651
3,Morning Energy Face Wash - Aqua Splash. Descri...,product\nWhitening Even Tone Body Lotion - UV ...,0.960858
4,Gas Lighter-Stainless Steel Aristo - Assorted ...,Softgrip Scissor (17cm). Description :Fackelma...,0.883930
...,...,...,...
134912,product\nMysore Pak Mysore Pak. Description...,product\nRasogolla Rasogolla. Description :...,0.920820
134913,Intensive Care Revitalizing Green Tea Body Lot...,Pigmentation Reducing Complex. Description :De...,0.944859
134914,product\nChia Seeds Chia Seeds. Description...,product\nCocoa Powder - Non-Alkalised Cocoa...,0.930789
134915,Palak 250g+Ladies Finger 500g+Cucumber 1kg+Cap...,Spring Onion - Chopped. Description :Spring on...,0.910863


In [13]:
combined = pd.concat([combination_df,mismatched_combination_df], ignore_index=True)
combined

,product_1,product_2,score
0,"Cold Extracted Juice - Basics, Guava. Descript...",Juice - Orange Oomph. Description :B Natural M...,0.948012
1,Dishwasher Rinse Aid forCrystal Shine. Descrip...,product\nGerm Kill Spray Germ Kill Spray. D...,0.928530
2,Pulse Deodorant. Description :The World’s # 1 ...,product\nCool Hold Styling Hair Gel Cool Ho...,0.891651
3,Morning Energy Face Wash - Aqua Splash. Descri...,product\nWhitening Even Tone Body Lotion - UV ...,0.960858
4,Gas Lighter-Stainless Steel Aristo - Assorted ...,Softgrip Scissor (17cm). Description :Fackelma...,0.883930
...,...,...,...
217576,Green Tea - Pure Original. Description :Tetley...,Organic Sattu Maavu Multi-Grain Millets Porrid...,0.196773
217577,Ayurvedic Massage Oil - Dhanwantharam Oil. Des...,Green Tea - Pure Original. Description :Tetley...,0.175249
217578,Green Tea - Pure Original. Description :Tetley...,Popular Aluminium Outer Lid Pressure Cooker (1...,0.156526
217579,Beetroot - Diced. Description :These edible ru...,United Dreams Go Far Deodorant. Description :T...,0.145101


In [14]:
combined["score"][213]

0.9058575846850216

In [15]:
combined = combined.rename(columns = {'product_1':'sentence1', 'product_2':'sentence2'})

In [16]:
combined["sentence1"][213]

'Sanitary Pads - Dry-Max Ultra-Dry, with Wings. Description :Dry cover ultra napkin for best day time protection.Dry-Max Cover for the best dry feel throughout the day.Odour Control System with natural plant extracts helps prevent odour for a fresh and confident feeling.Super Lock Pockets with Gel Core that rapidly locks-in even the heaviest flow.  For Beauty tips, tricks & more visitÃ‚\xa0https://bigbasket.blog/!'

In [17]:
from datasets import Dataset
hf = Dataset.from_pandas(combined)

In [18]:
hf

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 217581
})

In [19]:
from datasets import Dataset, DatasetDict

train_test_split = hf.train_test_split(test_size=0.2, seed=42)

test_eval_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': test_eval_split['train'],
    'eval': test_eval_split['test']
})
split_dataset['train'].save_to_disk('hf_dataset_train')
split_dataset['test'].save_to_disk('hf_dataset_test')
split_dataset['eval'].save_to_disk('hf_dataset_eval')

train_dataset = Dataset.load_from_disk('hf_dataset_train')
print(train_dataset[:5])


Saving the dataset (0/1 shards):   0%|          | 0/174064 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/21758 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/21759 [00:00<?, ? examples/s]

{'sentence1': ['Oil Clear Mud Face Pack. Description :Himalaya Oil Clear Mud Face Pack Rejuvenate your dead skin with Himalaya Oil Clear Mud Face Pack. This herbal formulation deep cleanses facial skin and clears clogged pores by absorbing excess oil and removing impurities. It helps maintain the natural pH of the skin and has deep cleansing and detoxifying properties, leaving the skin cleansed and revitalized. Fullers Earth removes deep-seated dirt and pollutants. It absorbs oil, clears clogged pores and blemishes and helps remove dead skin. Fullers Earth also helps lighten tanned skin caused by UV rays.!', 'Essence - Butter Scotch. Description :Concentrate Butterscotch Essence For Sauces, Desserts, Baking And Cakes.Butterscotch Adds A Luscious Flavor Note To Mochas, Lattes And Other Hot, Frozen And Chilled Drinks.!', 'Marie Light Biscuit - Vita Orange. Description :Sunfeast Marie Light orange offer crisp & light biscuits completed with the choicest golden grains of sun-ripened orange

In [20]:
eval_dataset = Dataset.load_from_disk('hf_dataset_eval')
eval_dataset['sentence2'][1]

'Erina - Coat Cleanser. Description :Action : Dandruff control : Erina prevents the formation of dandruff on your pets skin and hair coat. Antimicrobial : Its antiseptic and antibacterial cleansing eliminates germs and improves overall skin hygiene. Erina protects the body against commonly found pathogens that cause itching and bacterial infections. Deodorant : Erinas deodorizing properties eliminate foul odor. Indications : For controlling dandruff in the hair coat. Prevention and management of pruritus (itching) and pyoder(superficial bacterial infection). Used in routine bathing as a cleanser to maintain a healthy coat.!'

In [50]:
eval_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 21759
})

In [21]:
test_dataset = Dataset.load_from_disk('hf_dataset_test')

In [22]:
test_dataset['sentence2'][1]

'IMPORTED - 3Ply Kitchen Towel Super Absorbent Paper Tissue Roll. Description :Selpak Kitchen Towels are your biggest supporter in creating a hygienic and healthy environment for your family! 3-ply Selpak Paper Towels are highly durable and the ridges provide super absorbency. Moreover, you can use only as much as you need with the practical half-sized sheets. Each pack has four rolls. The packaging is made of biodegradable plastic which shows that we are committed to the environment. The product is made of 100% virgin cellulose which is naturally bright without the use of any chemicals.!'

In [23]:
model = SentenceTransformer("BAAI/bge-base-en", trust_remote_code=True)

In [64]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.training_args import BatchSamplers

loss = CosineSimilarityLoss(model)

args = SentenceTransformerTrainingArguments(
    output_dir="bge-small-ft-retail",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  
    bf16=True,  # Set to True if you have a GPU that supports BF16
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="bge-small-ft-retail",  
)

In [65]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    name="bge-test",
)

In [66]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
)

In [67]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.049100,0.039325
200,0.042700,0.033304
300,0.037300,0.025681
400,0.029400,0.018843
500,0.021200,0.016866
600,0.017400,0.013119
700,0.014500,0.012322
800,0.012500,0.009375
900,0.010900,0.010340
1000,0.010200,0.008602


TrainOutput(global_step=8160, training_loss=0.006283525598472824, metrics={'train_runtime': 14295.7704, 'train_samples_per_second': 36.528, 'train_steps_per_second': 0.571, 'total_flos': 0.0, 'train_loss': 0.006283525598472824, 'epoch': 3.0})

In [62]:
torch.cuda.empty_cache()

In [68]:
model.save_pretrained("models/bge-final-small-retail")

In [ ]:
model.push_to_hub("bge-final-small-retail-v2")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]